### Importamos las librerías necesarias para realizar web scraping

In [2]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

### Descargamos los datos del enlace: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, por medio de BeatifulSoup

In [3]:
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = requests.get(URL)
soup = BeautifulSoup(page.content, "html.parser")

### Se extrae la tabla que contiene solamente los datos de códigos postales de Toronto

In [4]:
table = soup.find('table')

In [5]:
file=[]
ncol=list(range(0,9))

for i in ncol :
    
    for row in table.find_all('tr'): # in html table row is represented by the table
    # Get all columns in each row.
        cols = row.find_all('p') # in html a column is represented by the tag td or p
        file.append(cols[i].getText())

    # convert to dataframe:
df= pd.DataFrame(file)

### Luego, configuramos el datafreme solicitado en la tarea

In [6]:
df["Postal Code"] = df[0].str[0:3]
df["Borough"] = df[0].str[3:].str.split("(", n=1, expand=True)[0]
df["Neigh"] = df[0].str.rsplit("(", n=2, expand=True)[1].str.split(")", n=1, expand=True)[0].fillna("")
df["Neighbourhood"] = df["Neigh"].str.replace(" / ", ",")

In [7]:
postalcode_list=df[(df["Borough"]!="Not assigned\n")]
postalcode_list=postalcode_list.drop([0, 'Neigh'],axis=1)

In [8]:
postalcode_list.head()

,Postal Code,Borough,Neighbourhood
1,M1B,Scarborough,"Malvern,Rouge"
2,M1C,Scarborough,"Rouge Hill,Port Union,Highland Creek"
3,M1E,Scarborough,"Guildwood,Morningside,West Hill"
4,M1G,Scarborough,Woburn
5,M1H,Scarborough,Cedarbrae


In [9]:
postalcode_list.shape

(103, 3)

### Descargamos el archivo de internet con el enlace dado, que nos proporciona coordenadas geográficas

In [10]:
# URL del archivo CSV en internet
url = 'https://cocl.us/Geospatial_data'

Geospatial_Coordinates=pd.read_csv(url)

### Anexamos las coordenadas geográficas a nuestro dataframe por medio del código postal

In [11]:
coord_list= pd.merge(postalcode_list, Geospatial_Coordinates, on="Postal Code")

coord_list = coord_list.drop("Postal Code", axis=1)

coord_list.head()

,Borough,Neighbourhood,Latitude,Longitude
0,Scarborough,"Malvern,Rouge",43.806686,-79.194353
1,Scarborough,"Rouge Hill,Port Union,Highland Creek",43.784535,-79.160497
2,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,Scarborough,Woburn,43.770992,-79.216917
4,Scarborough,Cedarbrae,43.773136,-79.239476


### Ubicamos los barrios de Toronto en el mapa

In [12]:
from geopy.geocoders import Nominatim

address = 'TORONTO,CA'
geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [13]:
toronto_data = coord_list[coord_list['Borough'].str.contains('Toronto')].reset_index(drop=True)
toronto_data.groupby("Borough").size().sort_values(ascending=False)

Borough
Downtown Toronto                                                17
Central Toronto                                                  9
West Toronto                                                     6
East Toronto                                                     4
Downtown TorontoStn A PO Boxes25 The Esplanade                   1
East TorontoBusiness reply mail Processing Centre969 Eastern     1
East YorkEast Toronto                                            1
dtype: int64

In [14]:
! pip install folium

import folium

# crear un mapa de Manhattan usando los valores de latitud y longitud
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# añadir los marcadores al mapa
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Definimos las credenciales de la API de Foursquare y la versión en la que vamos a trabajar

In [15]:
CLIENT_ID = '2OF3KKCYFHP50KM33SYTWRWORAU5A2IWLQGYH2K3NRPGTTMU' # su ID de Foursquare
CLIENT_SECRET = 'TSQBJ2TIQLG5KFUSS234GQO5KGMFVWL5FVKWVV5KYVQJUPXB' # su Secreto de Cliente de Foursquare
#VERSION = '20180604'
VERSION = '20230417'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 2OF3KKCYFHP50KM33SYTWRWORAU5A2IWLQGYH2K3NRPGTTMU
CLIENT_SECRET:TSQBJ2TIQLG5KFUSS234GQO5KGMFVWL5FVKWVV5KYVQJUPXB


In [17]:
toronto_data.loc[0, 'Neighbourhood']

'The Beaches'

In [18]:
neighborhood_latitude = toronto_data.loc[0, 'Latitude'] # latitud del barrio 
neighborhood_longitude = toronto_data.loc[0, 'Longitude'] # longitud del barrio

neighborhood_name = toronto_data.loc[0, 'Neighbourhood'] # nombre del barrio

print('Latitude and longitude values of {} are {},{}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of The Beaches are 43.6763574,-79.2930312.


In [19]:
LIMIT= 50
radius= 500
url = "https://api.foursquare.com/v3/places/search?ll={}%2C{}&radius={}&limit={}".format(neighborhood_latitude, neighborhood_longitude, radius, LIMIT)
url

'https://api.foursquare.com/v3/places/search?ll=43.6763574%2C-79.2930312&radius=500&limit=50'

In [20]:
#url = "https://api.foursquare.com/v3/places/search?ll=43.6763574%2C%20-79.2930312&radius=500&limit=50"
#url = "https://api.foursquare.com/v3/places/search?ll={}%2C%20{}&radius={}&limit={}".format(neighborhood_latitude, neighborhood_longitude, radius, LIMIT)

headers = {
    "accept": "application/json",
    "Authorization": "fsq3+btNZJiWH0LtffL3n4zu2NiE6iThx9A752cLEJs5q1U="
}
#response = requests.get(url, headers=headers)
#results = requests.get(url, headers=headers)
results = requests.get(url, headers=headers).json()
results
#print(results.text)

{'results': [{'fsq_id': '4bd461bc77b29c74a07d9282',
   'categories': [{'id': 16019,
     'name': 'Hiking Trail',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/hikingtrail_',
      'suffix': '.png'}}],
   'chains': [],
   'distance': 89,
   'geocodes': {'main': {'latitude': 43.676676, 'longitude': -79.294126},
    'roof': {'latitude': 43.676676, 'longitude': -79.294126}},
   'link': '/v3/places/4bd461bc77b29c74a07d9282',
   'location': {'country': 'CA',
    'cross_street': 'Queen St.',
    'formatted_address': 'Queen St., Toronto ON',
    'locality': 'Toronto',
    'region': 'ON'},
   'name': 'Glen Manor Ravine',
   'related_places': {},
   'timezone': 'America/Toronto'},
  {'fsq_id': '4dbc8fe96a23e294ba3237bd',
   'categories': [{'id': 16032,
     'name': 'Park',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/park_',
      'suffix': '.png'}}],
   'chains': [],
   'distance': 176,
   'geocodes': {'main': {'latitude': 43.67

In [22]:
from pandas.io.json import json_normalize 
venues = results['results']
nearby_venues = json_normalize(venues) # objeto JSON

C:\Users\INFORMEGA\AppData\Local\Temp/ipykernel_3704/18415710.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # objeto JSON


In [23]:
nearby_venues.head()

,fsq_id,categories,chains,distance,link,name,timezone,geocodes.main.latitude,geocodes.main.longitude,geocodes.roof.latitude,...,location.cross_street,location.formatted_address,location.locality,location.region,location.address,location.postcode,related_places.children,geocodes.drop_off.latitude,geocodes.drop_off.longitude,location.address_extended
0,4bd461bc77b29c74a07d9282,"[{'id': 16019, 'name': 'Hiking Trail', 'icon':...",[],89,/v3/places/4bd461bc77b29c74a07d9282,Glen Manor Ravine,America/Toronto,43.676676,-79.294126,43.676676,...,Queen St.,"Queen St., Toronto ON",Toronto,ON,NaN,NaN,NaN,NaN,NaN,NaN
1,4dbc8fe96a23e294ba3237bd,"[{'id': 16032, 'name': 'Park', 'icon': {'prefi...",[],176,/v3/places/4dbc8fe96a23e294ba3237bd,Glen Stewart Park,America/Toronto,43.674403,-79.294516,43.674403,...,,"131 Glen Manor Dr, Toronto ON",Toronto,ON,131 Glen Manor Dr,NaN,NaN,NaN,NaN,NaN
2,0bb2d5a345d02032b733f0b6,"[{'id': 11006, 'name': 'Audiovisual Service', ...",[],122,/v3/places/0bb2d5a345d02032b733f0b6,Santon Audio,NaN,43.677003,-79.294270,43.677003,...,NaN,"4 Jim Wilson Lane, Toronto ON M4E 0C1",Toronto,ON,4 Jim Wilson Lane,M4E 0C1,NaN,NaN,NaN,NaN
3,4b9c6e76c8ea40047bca6aa7,"[{'id': 11041, 'name': 'Film Studio', 'icon': ...",[],212,/v3/places/4b9c6e76c8ea40047bca6aa7,Verite Films Inc,NaN,43.675897,-79.295593,43.675897,...,NaN,"15 Beaufort Rd, Toronto ON M4E 1M6",Toronto,ON,15 Beaufort Rd,M4E 1M6,NaN,NaN,NaN,NaN
4,4ad4c062f964a52011f820e3,"[{'id': 17069, 'name': 'Grocery Store / Superm...",[],470,/v3/places/4ad4c062f964a52011f820e3,The Big Carrot Beach Community Market,America/Toronto,43.678841,-79.297600,43.678841,...,NaN,"125 Southwood Dr, Toronto ON M4E 0B8",Toronto,ON,125 Southwood Dr,M4E 0B8,"[{'fsq_id': '4bd1db4e046076b0f1a57271', 'name'...",NaN,NaN,NaN


In [24]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['categories.name']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
nearby_venues['categories'] = nearby_venues.apply(get_category_type, axis=1)
nearby_venues.head()

,fsq_id,categories,chains,distance,link,name,timezone,geocodes.main.latitude,geocodes.main.longitude,geocodes.roof.latitude,...,location.cross_street,location.formatted_address,location.locality,location.region,location.address,location.postcode,related_places.children,geocodes.drop_off.latitude,geocodes.drop_off.longitude,location.address_extended
0,4bd461bc77b29c74a07d9282,Hiking Trail,[],89,/v3/places/4bd461bc77b29c74a07d9282,Glen Manor Ravine,America/Toronto,43.676676,-79.294126,43.676676,...,Queen St.,"Queen St., Toronto ON",Toronto,ON,NaN,NaN,NaN,NaN,NaN,NaN
1,4dbc8fe96a23e294ba3237bd,Park,[],176,/v3/places/4dbc8fe96a23e294ba3237bd,Glen Stewart Park,America/Toronto,43.674403,-79.294516,43.674403,...,,"131 Glen Manor Dr, Toronto ON",Toronto,ON,131 Glen Manor Dr,NaN,NaN,NaN,NaN,NaN
2,0bb2d5a345d02032b733f0b6,Audiovisual Service,[],122,/v3/places/0bb2d5a345d02032b733f0b6,Santon Audio,NaN,43.677003,-79.294270,43.677003,...,NaN,"4 Jim Wilson Lane, Toronto ON M4E 0C1",Toronto,ON,4 Jim Wilson Lane,M4E 0C1,NaN,NaN,NaN,NaN
3,4b9c6e76c8ea40047bca6aa7,Film Studio,[],212,/v3/places/4b9c6e76c8ea40047bca6aa7,Verite Films Inc,NaN,43.675897,-79.295593,43.675897,...,NaN,"15 Beaufort Rd, Toronto ON M4E 1M6",Toronto,ON,15 Beaufort Rd,M4E 1M6,NaN,NaN,NaN,NaN
4,4ad4c062f964a52011f820e3,Grocery Store / Supermarket,[],470,/v3/places/4ad4c062f964a52011f820e3,The Big Carrot Beach Community Market,America/Toronto,43.678841,-79.297600,43.678841,...,NaN,"125 Southwood Dr, Toronto ON M4E 0B8",Toronto,ON,125 Southwood Dr,M4E 0B8,"[{'fsq_id': '4bd1db4e046076b0f1a57271', 'name'...",NaN,NaN,NaN


In [25]:
filtered_columns = ['name', 'categories', 'geocodes.main.latitude', 'geocodes.main.longitude']
nearby_venues =nearby_venues.loc[:, filtered_columns]
nearby_venues.head()

,name,categories,geocodes.main.latitude,geocodes.main.longitude
0,Glen Manor Ravine,Hiking Trail,43.676676,-79.294126
1,Glen Stewart Park,Park,43.674403,-79.294516
2,Santon Audio,Audiovisual Service,43.677003,-79.294270
3,Verite Films Inc,Film Studio,43.675897,-79.295593
4,The Big Carrot Beach Community Market,Grocery Store / Supermarket,43.678841,-79.297600


In [26]:
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,latitude,longitude
0,Glen Manor Ravine,Hiking Trail,43.676676,-79.294126
1,Glen Stewart Park,Park,43.674403,-79.294516
2,Santon Audio,Audiovisual Service,43.677003,-79.294270
3,Verite Films Inc,Film Studio,43.675897,-79.295593
4,The Big Carrot Beach Community Market,Grocery Store / Supermarket,43.678841,-79.297600


In [27]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # crear la URL de solicitud de API
        url = "https://api.foursquare.com/v3/places/search?ll={}%2C{}&radius={}&limit={}".format(lat, lng, radius, LIMIT)
        headers = {
        "accept": "application/json",
        "Authorization": "fsq3+btNZJiWH0LtffL3n4zu2NiE6iThx9A752cLEJs5q1U="
        }
            
        # solicitud GET
        results = requests.get(url, headers=headers).json()
        venues = results['results']

        
        # regresa solo información relevante de cada sitio cercano
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['name'], 
            v['geocodes']['main']['latitude'], 
            v['geocodes']['main']['longitude'],  
            (v['categories'][0]['name'] if len(v['categories']) > 0 else None)) for v in venues])
       


    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Obtenemos las calles de Toronto

In [28]:
nearby_venues

,name,categories,latitude,longitude
0,Glen Manor Ravine,Hiking Trail,43.676676,-79.294126
1,Glen Stewart Park,Park,43.674403,-79.294516
2,Santon Audio,Audiovisual Service,43.677003,-79.294270
3,Verite Films Inc,Film Studio,43.675897,-79.295593
4,The Big Carrot Beach Community Market,Grocery Store / Supermarket,43.678841,-79.297600
5,Beacher Cafe,Diner,43.672055,-79.291248
6,Maple Hosiery Mills,None,43.674353,-79.292077
7,Tscc 1930,None,43.674117,-79.293022
8,Coysh School of Dance,Dance Studio,43.677452,-79.295838
9,Williamson Road Playground,Playground,43.674602,-79.297214


In [30]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighbourhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

The Beaches
The Danforth  East
The Danforth West,Riverdale
India Bazaar,The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park,Summerhill East
Summerhill West,Rathnelly,South Hill,Forest Hill SE,Deer Park
Rosedale
St. James Town,Cabbagetown
Church and Wellesley
Regent Park,Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond,Adelaide,King
Harbourfront East,Union Station,Toronto Islands
Toronto Dominion Centre,Design Exchange
Commerce Court,Victoria Hotel
Roselawn
Forest Hill North & West
The Annex,North Midtown,Yorkville
University of Toronto,Harbord
Kensington Market,Chinatown,Grange Park
CN Tower,King and Spadina,Railway Lands,Harbourfront West,Bathurst Quay,South Niagara,Island airport
Enclave of M5E
First Canadian Place,Underground city
Christie
Dufferin,Dovercourt Village
Little Portugal,Trinity
Brockton,Parkdale Village,Exhibition Place
High Park,The Junction South
Parkdale,Roncesvalles
Runn

In [31]:
print(toronto_venues.shape)
toronto_venues.head()

(1791, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676676,-79.294126,Hiking Trail
1,The Beaches,43.676357,-79.293031,Glen Stewart Park,43.674403,-79.294516,Park
2,The Beaches,43.676357,-79.293031,Santon Audio,43.677003,-79.294270,Audiovisual Service
3,The Beaches,43.676357,-79.293031,Verite Films Inc,43.675897,-79.295593,Film Studio
4,The Beaches,43.676357,-79.293031,The Big Carrot Beach Community Market,43.678841,-79.297600,Grocery Store / Supermarket


### Exploramos los sitios de cada Barrio

In [32]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,50,50,50,50,50,50
"Brockton,Parkdale Village,Exhibition Place",50,50,50,50,50,47
"CN Tower,King and Spadina,Railway Lands,Harbourfront West,Bathurst Quay,South Niagara,Island airport",13,13,13,13,13,9
Central Bay Street,50,50,50,50,50,50
Christie,50,50,50,50,50,47
Church and Wellesley,50,50,50,50,50,50
"Commerce Court,Victoria Hotel",50,50,50,50,50,50
Davisville,50,50,50,50,50,48
Davisville North,50,50,50,50,50,43


In [33]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 320 uniques categories.


### Procedemos a analizar cada barrio

In [34]:
# codificación
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# añadir la columna de barrio de regreso al dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# mover la columna de barrio a la primer columna
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,ATM,Accounting and Bookkeeping Service,Advertising Agency,Afghan Restaurant,African Restaurant,American Restaurant,Antique Store,Architecture Firm,Art Gallery,...,Vegan and Vegetarian Restaurant,Video Games Store,Video Store,Vietnamese Restaurant,Vintage and Thrift Store,Website Designer,Wholesaler,Wine Bar,Women's Store,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
toronto_onehot.shape

(1791, 320)

In [36]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,ATM,Accounting and Bookkeeping Service,Advertising Agency,Afghan Restaurant,African Restaurant,American Restaurant,Antique Store,Architecture Firm,Art Gallery,...,Vegan and Vegetarian Restaurant,Video Games Store,Video Store,Vietnamese Restaurant,Vintage and Thrift Store,Website Designer,Wholesaler,Wine Bar,Women's Store,Yoga Studio
0,Berczy Park,0.00,0.000000,0.00,0.00,0.02,0.02,0.00,0.000000,0.02,...,0.02,0.00,0.00,0.00,0.000000,0.00000,0.00,0.00,0.00,0.00
1,"Brockton,Parkdale Village,Exhibition Place",0.00,0.000000,0.02,0.00,0.00,0.00,0.00,0.000000,0.00,...,0.00,0.00,0.00,0.00,0.000000,0.00000,0.00,0.00,0.00,0.00
2,"CN Tower,King and Spadina,Railway Lands,Harbou...",0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,...,0.00,0.00,0.00,0.00,0.000000,0.00000,0.00,0.00,0.00,0.00
3,Central Bay Street,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,...,0.00,0.00,0.00,0.00,0.000000,0.00000,0.00,0.00,0.00,0.00
4,Christie,0.00,0.000000,0.02,0.00,0.00,0.02,0.02,0.020000,0.00,...,0.00,0.00,0.00,0.00,0.000000,0.00000,0.00,0.02,0.00,0.00
5,Church and Wellesley,0.02,0.000000,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,...,0.00,0.00,0.00,0.00,0.000000,0.00000,0.00,0.00,0.00,0.00
6,"Commerce Court,Victoria Hotel",0.00,0.000000,0.02,0.00,0.00,0.02,0.00,0.000000,0.00,...,0.00,0.00,0.00,0.00,0.000000,0.00000,0.00,0.00,0.00,0.00
7,Davisville,0.00,0.000000,0.00,0.00,0.00,0.02,0.02,0.000000,0.00,...,0.00,0.00,0.00,0.00,0.020000,0.00000,0.02,0.00,0.00,0.00
8,Davisville North,0.00,0.000000,0.02,0.00,0.00,0.00,0.00,0.000000,0.00,...,0.00,0.00,0.00,0.00,0.000000,0.04000,0.00,0.00,0.00,0.00
9,"Dufferin,Dovercourt Village",0.02,0.020000,0.02,0.00,0.00,0.02,0.00,0.000000,0.00,...,0.00,0.00,0.00,0.00,0.000000,0.00000,0.00,0.00,0.00,0.00


In [37]:
toronto_grouped.shape

(39, 320)

### Mostramos los 5 lugares más comunes de cada barrio

In [38]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                 venue  freq
0                  Pub  0.06
1          Coffee Shop  0.06
2             Beer Bar  0.04
3  Fish and Chips Shop  0.04
4           Night Club  0.04


----Brockton,Parkdale Village,Exhibition Place----
                       venue  freq
0                 Hair Salon  0.06
1  Telecommunication Service  0.04
2    Computer Repair Service  0.04
3                       Café  0.04
4              Design Studio  0.04


----CN Tower,King and Spadina,Railway Lands,Harbourfront West,Bathurst Quay,South Niagara,Island airport----
                         venue  freq
0     Home Improvement Service  0.08
1               Car Dealership  0.08
2            Convenience Store  0.08
3        Sports and Recreation  0.08
4  Historic and Protected Site  0.08


----Central Bay Street----
                           venue  freq
0  Cafes, Coffee, and Tea Houses  0.12
1                    Coffee Shop  0.08
2                           Deli  0.06
3                        

### Creamos una función para ordenar los lugares y lugo creamos un dataframe con los 10 lugares más populares de cada barrrio

In [39]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [40]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# crear las columnas acorde al numero de sitios populares
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# crear un nuevo dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Pub,Coffee Shop,Beer Bar,Fish and Chips Shop,Night Club,Sushi Restaurant,Farmers' Market,Drugstore,Park,Butcher
1,"Brockton,Parkdale Village,Exhibition Place",Hair Salon,Telecommunication Service,Computer Repair Service,Café,Design Studio,Photography Studio,Laundry Service,Coffee Shop,Grocery Store / Supermarket,Board Store
2,"CN Tower,King and Spadina,Railway Lands,Harbou...",Home Improvement Service,Car Dealership,Convenience Store,Sports and Recreation,Historic and Protected Site,Property Management Office,Landmarks and Outdoors,Sculpture Garden,Harbor / Marina,Nutritionist
3,Central Bay Street,"Cafes, Coffee, and Tea Houses",Coffee Shop,Deli,Café,Pizzeria,Diner,Fast Food Restaurant,Japanese Restaurant,Spa,Mediterranean Restaurant
4,Christie,Grocery Store / Supermarket,Automotive Repair Shop,Electrician,Hardware Store,Hair Salon,Café,Restaurant,Management Consultant,Massage Clinic,Computers and Electronics Retail


### Creamos nuestro modelo de Clustering mediante algoritmo de K-means con K=5

In [42]:
from sklearn.cluster import KMeans
# establecer el número de agrupaciones
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# ejecutar k-means
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# revisar las etiquetas de las agrupaciones generadas para cada fila del dataframe
kmeans.labels_[0:10] 

C:\Users\INFORMEGA\AppData\Local\Temp/ipykernel_3704/1867066911.py:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)


array([1, 3, 4, 1, 3, 2, 2, 3, 3, 3])

In [50]:
# añadir etiquetas
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# juntar toronto_grouped con toronto_data 
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

toronto_merged.head() # revisar las ultimas columnas

,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,The Beaches,43.676357,-79.293031,3,Interior Designer,Hair Salon,Park,Diner,Organization,Dance Studio,Playground,Boutique,Lingerie Store,Grocery Store / Supermarket
1,East YorkEast Toronto,The Danforth East,43.685347,-79.338106,3,Business and Professional Services,Convenience Store,Park,Plumber,Event Service,Pet Sitting and Boarding Service,Health and Beauty Service,Barbershop,Bathroom Contractor,College and University
2,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188,2,Restaurant,Bakery,Ice Cream Parlor,Greek Restaurant,Diner,Italian Restaurant,Bookstore,Hair Salon,Retail,Café
3,East Toronto,"India Bazaar,The Beaches West",43.668999,-79.315572,1,Pizzeria,Café,Pet Grooming Service,Architecture Firm,Bicycle Store,Park,Convenience Store,Furniture and Home Store,Business and Professional Services,Burrito Restaurant
4,East Toronto,Studio District,43.659526,-79.340923,2,Bar,Drugstore,Retail,Nail Salon,Italian Restaurant,Coffee Shop,Bakery,Tattoo Parlor,Fast Food Restaurant,Hardware Store


In [51]:
neighborhoods_venues_sorted.head()

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1,Berczy Park,Pub,Coffee Shop,Beer Bar,Fish and Chips Shop,Night Club,Sushi Restaurant,Farmers' Market,Drugstore,Park,Butcher
1,3,"Brockton,Parkdale Village,Exhibition Place",Hair Salon,Telecommunication Service,Computer Repair Service,Café,Design Studio,Photography Studio,Laundry Service,Coffee Shop,Grocery Store / Supermarket,Board Store
2,4,"CN Tower,King and Spadina,Railway Lands,Harbou...",Home Improvement Service,Car Dealership,Convenience Store,Sports and Recreation,Historic and Protected Site,Property Management Office,Landmarks and Outdoors,Sculpture Garden,Harbor / Marina,Nutritionist
3,1,Central Bay Street,"Cafes, Coffee, and Tea Houses",Coffee Shop,Deli,Café,Pizzeria,Diner,Fast Food Restaurant,Japanese Restaurant,Spa,Mediterranean Restaurant
4,3,Christie,Grocery Store / Supermarket,Automotive Repair Shop,Electrician,Hardware Store,Hair Salon,Café,Restaurant,Management Consultant,Massage Clinic,Computers and Electronics Retail


### Visualizamos en el mapa los clusters

In [53]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# crear mapa
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# establecer el esquema de color para las agrupaciones
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# añadir marcadores al mapa
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Definimos las categorías de nuestras agrupaciones

### Cluster 1. Parques

In [54]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Lawrence Park,Business and Strategy Consulting Office,Architecture Firm,Publisher,Management Consultant,Organization,Tennis Club,Legal Service,Spa,Bakery,Park
9,"Moore Park,Summerhill East",Business and Strategy Consulting Office,Tennis Court,Business and Professional Services,Financial Planner,Architecture Firm,Playground,Machine Shop,Organization,Real Estate Agency,Retail
23,Roselawn,Business and Strategy Consulting Office,Construction,Business and Professional Services,Computer Repair Service,Mover,"Shipping, Freight, and Material Transportation...",Candy Store,Nutritionist,General Contractor,Electrician
24,Forest Hill North & West,Business and Professional Services,General Contractor,Home Improvement Service,"Heating, Ventilating and Air Conditioning Cont...",Campground,Carpet and Flooring Contractor,Caterer,Management Consultant,Gift Store,Event Service
36,"Parkdale,Roncesvalles",Retail,Business and Strategy Consulting Office,Bank,Computer Repair Service,Bar,Tennis Court,Telecommunication Service,Painter,Management Consultant,Hair Salon


### Cluster 2. Barrios del Centro

In [56]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,"India Bazaar,The Beaches West",Pizzeria,Café,Pet Grooming Service,Architecture Firm,Bicycle Store,Park,Convenience Store,Furniture and Home Store,Business and Professional Services,Burrito Restaurant
15,"Garden District, Ryerson",Fast Food Restaurant,Department Store,Café,Bookstore,Burger Joint,Coffee Shop,Clothing Store,Beer Bar,Karaoke Bar,Pizzeria
16,St. James Town,Diner,Café,Pizzeria,Coffee Shop,BBQ Joint,Asian Restaurant,Japanese Restaurant,Sports Bar,Bistro,Hair Salon
17,Berczy Park,Pub,Coffee Shop,Beer Bar,Fish and Chips Shop,Night Club,Sushi Restaurant,Farmers' Market,Drugstore,Park,Butcher
18,Central Bay Street,"Cafes, Coffee, and Tea Houses",Coffee Shop,Deli,Café,Pizzeria,Diner,Fast Food Restaurant,Japanese Restaurant,Spa,Mediterranean Restaurant
20,"Harbourfront East,Union Station,Toronto Islands",Pizzeria,Park,Coffee Shop,Bar,Deli,Sports Bar,Plaza,Café,Music Venue,Salad Restaurant
25,"The Annex,North Midtown,Yorkville",Café,Laundromat,Playground,Deli,Coffee Shop,Park,Dry Cleaner,Music School,Miscellaneous Store,Furniture and Home Store
26,"University of Toronto,Harbord",Pizzeria,Café,Bookstore,Bank,Hot Dog Joint,Bakery,College and University,College Administrative Building,Coffee Shop,ATM
27,"Kensington Market,Chinatown,Grange Park",Café,Chinese Restaurant,Vietnamese Restaurant,Bakery,Barbershop,Bar,Jewelry Store,Fish and Chips Shop,Mexican Restaurant,Dive Bar
29,Enclave of M5E,Pub,Farmers' Market,Bistro,Music Venue,Night Club,Fish and Chips Shop,Coffee Shop,Beer Bar,Drugstore,American Restaurant


### Cluster 3. Suburbio Oeste

In [58]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,"The Danforth West,Riverdale",Restaurant,Bakery,Ice Cream Parlor,Greek Restaurant,Diner,Italian Restaurant,Bookstore,Hair Salon,Retail,Café
4,Studio District,Bar,Drugstore,Retail,Nail Salon,Italian Restaurant,Coffee Shop,Bakery,Tattoo Parlor,Fast Food Restaurant,Hardware Store
12,"St. James Town,Cabbagetown",Pizzeria,Bakery,Restaurant,Massage Clinic,Pet Supplies Store,Convenience Store,Dance Studio,Chinese Restaurant,Grocery Store / Supermarket,Taco Restaurant
13,Church and Wellesley,Diner,Hair Salon,Tattoo Parlor,Thai Restaurant,Restaurant,Clothing Store,Pizzeria,Liquor Store,Bubble Tea Shop,Italian Restaurant
14,"Regent Park,Harbourfront",Coffee Shop,Restaurant,Car Dealership,Bakery,Automotive Repair Shop,Café,Park,Music Venue,Design Studio,Farmers' Market
19,"Richmond,Adelaide,King",Café,Restaurant,"Cafes, Coffee, and Tea Houses",Lounge,Deli,Salad Restaurant,Music Venue,Burger Joint,Sushi Restaurant,Pizzeria
21,"Toronto Dominion Centre,Design Exchange",Restaurant,"Cafes, Coffee, and Tea Houses",Bakery,Bank,Café,Deli,Coffee Shop,Burger Joint,Drugstore,Diner
22,"Commerce Court,Victoria Hotel",Café,"Cafes, Coffee, and Tea Houses",Restaurant,Bakery,Coffee Shop,Diner,Pub,Bank,Bistro,Steakhouse
30,"First Canadian Place,Underground city",Restaurant,Café,"Cafes, Coffee, and Tea Houses",Bakery,Burger Joint,Coffee Shop,Deli,Bank,Thai Restaurant,Diner


### Cluster 4. Suburbio Norteste

In [59]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,The Beaches,Interior Designer,Hair Salon,Park,Diner,Organization,Dance Studio,Playground,Boutique,Lingerie Store,Grocery Store / Supermarket
1,The Danforth East,Business and Professional Services,Convenience Store,Park,Plumber,Event Service,Pet Sitting and Boarding Service,Health and Beauty Service,Barbershop,Bathroom Contractor,College and University
6,Davisville North,Attorney / Law Office,Interior Designer,Playground,Public and Social Service,Community and Government,Tailor,Organization,Website Designer,Restaurant,General Contractor
7,North Toronto West,Clothing Store,Nail Salon,Park,Health and Beauty Service,Sushi Restaurant,Hair Salon,Retail,"Cafes, Coffee, and Tea Houses",Cosmetics Store,Spa
8,Davisville,Flower Store,Nail Salon,Health and Beauty Service,Italian Restaurant,Toy / Game Store,Pizzeria,New American Restaurant,Bookstore,Tailor,Hair Salon
10,"Summerhill West,Rathnelly,South Hill,Forest Hi...",Interior Designer,Sports and Recreation,Advertising Agency,Bagel Shop,Restaurant,Insurance Agency,Attorney / Law Office,"Cafes, Coffee, and Tea Houses",Café,Physician
11,Rosedale,Park,Interior Designer,Carpet and Flooring Contractor,Sports and Recreation,Business and Professional Services,Caterer,Real Estate Agency,Computer Repair Service,Property Management Office,Playground
31,Christie,Grocery Store / Supermarket,Automotive Repair Shop,Electrician,Hardware Store,Hair Salon,Café,Restaurant,Management Consultant,Massage Clinic,Computers and Electronics Retail
32,"Dufferin,Dovercourt Village",Fabric / Textiles Store,Liquor Store,Jewelry Store,Photographer,Sports and Recreation,Hair Salon,Automotive Repair Shop,Car Wash and Detail,Tailor,Media Agency
34,"Brockton,Parkdale Village,Exhibition Place",Hair Salon,Telecommunication Service,Computer Repair Service,Café,Design Studio,Photography Studio,Laundry Service,Coffee Shop,Grocery Store / Supermarket,Board Store


### Cluster 3. Isla central (aeródromo)

In [60]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
28,"CN Tower,King and Spadina,Railway Lands,Harbou...",Home Improvement Service,Car Dealership,Convenience Store,Sports and Recreation,Historic and Protected Site,Property Management Office,Landmarks and Outdoors,Sculpture Garden,Harbor / Marina,Nutritionist
